# Goal

The goal of this Lab is to demonstrate how to create a simple Agent using Pydantic AI.

# LLM Connection

We define the LLM

In [ ]:
from pydantic_settings import BaseSettings
import os

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    MODEL_NAME: str = "google-gla:gemini-2.5-flash"
    CONTEXT7_API_KEY: str
    class Config:
        #ignore extra fields
        extra = "ignore"
        env_file = ".env"

settings = Settings()
os.environ["GOOGLE_API_KEY"] = settings.GOOGLE_API_KEY

# Simple LLM Agent

Simple LLM call

In [2]:
from pydantic_ai import Agent
joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian, you tell jokes",
)
result = await joke_agent.run("Tell me a joke")
print(result.output)

Okay, here's one for you:

Why don't scientists trust atoms?

...Because they make up *everything*!


## Structured LLM Call

We define the Pydantic model for the structured output

In [3]:
from pydantic_ai import Agent
from pydantic import BaseModel

class joke_output(BaseModel):
    joke: str

joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian, you tell jokes",
                output_type=joke_output
)
result = await joke_agent.run("Tell me a joke")
print(result.output)

joke="Why don't scientists trust atoms? Because they make up everything!"


what if we want 4 jokes?

In [4]:
jokes_history = []
for i in range(4):
    result = await joke_agent.run("Tell me a joke")
    jokes_history.append(result.output.joke)

jokes_history

["Why don't scientists trust atoms? Because they make up everything!",
 "Why don't scientists trust atoms? Because they make up everything!",
 'Why did the computer go to the doctor? Because it had a virus!',
 "Why don't scientists trust atoms? Because they make up everything!"]

## Dependencies

Add the JokeDependencies to keep track of the jokes

In [5]:
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass

# We use dataclass because it allows us to add any object as a dependency, in this case a list of strings
@dataclass
class JokeDependencies:
    previous_jokes: list[str]

# Output defined by Pydantic
class joke_output(BaseModel):
    joke: str

joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian. You are given a list of previous jokes. You need to tell a new joke that is not in the list",
                deps_type=JokeDependencies,
                output_type=joke_output)

@joke_agent.system_prompt  
async def add_previous_jokes(ctx: RunContext[JokeDependencies]) -> str:
    return f"The previous jokes are {ctx.deps.previous_jokes}"

result = await joke_agent.run("Tell me a joke", deps=JokeDependencies(previous_jokes=[]))
print(result.output)

joke="Why don't scientists trust atoms? Because they make up everything!"


Now, what if we want 4 jokes?


In [6]:
jokes_history = JokeDependencies(previous_jokes=[])
for i in range(4):
    result = await joke_agent.run("Tell me a joke", deps=jokes_history)
    jokes_history.previous_jokes.append(result.output.joke)

jokes_history.previous_jokes
    

["Why don't scientists trust atoms? Because they make up everything!",
 'Why did the scarecrow win an award? Because he was outstanding in his field!',
 'What do you call a fake noodle? An impasta!',
 'What do you call a snowman with a six-pack? An abdominal snowman!']

# Agent with Tools

Definition of tools

In [7]:
from pydantic_settings import BaseSettings
from random import randint
from pydantic_ai import Agent

# We use dataclass because it allows us to add any object as a dependency, in this case a list of strings
@dataclass
class JokeDependencies:
    previous_jokes: list[str]

# Output defined by Pydantic
class joke_output(BaseModel):
    joke: str

joke_agent = Agent(settings.MODEL_NAME,
                instructions="You are a comedian. You are given a list of previous jokes. You need to tell a new joke that is not in the list",
                deps_type=JokeDependencies,
                output_type=joke_output)

@joke_agent.system_prompt  
async def add_previous_joke(ctx: RunContext[JokeDependencies]) -> str:
    return f"The previous jokes are {ctx.deps.previous_jokes}"

async def tell_a_joke(previous_jokes: list[str]=[]) -> str:
    """Creates a new joke, provide a list of previous jokes, if there are no previous jokes, just provide an empty list"""
    result = await joke_agent.run("Tell me a joke", deps=JokeDependencies(previous_jokes=previous_jokes))
    return result.output.joke

def random_number(min_val: int = 1, max_val: int = 100) -> int:
    """Generates a random number in a given range."""
    return randint(min_val, max_val)

## Agent that uses tools

We define the agent with the new output model and the tools.

In [8]:
class PoemOutput(BaseModel):
    jokes: list[str]
    numbers: list[int]
    poem: str

agent = Agent(settings.MODEL_NAME, tools=[tell_a_joke, random_number], output_type=PoemOutput)

# Run agent
nodes = []
async with agent.iter(
    "Please give me 3 random jokes and 5 random numbers. Then write a small poem about them.",
) as agent_run:
    async for node in agent_run:
        # Each node represents a step in the agent's execution
        print(node)
        nodes.append(node)

UserPromptNode(user_prompt='Please give me 3 random jokes and 5 random numbers. Then write a small poem about them.', instructions=None, instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})
ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Please give me 3 random jokes and 5 random numbers. Then write a small poem about them.', timestamp=datetime.datetime(2025, 9, 3, 22, 19, 7, 348015, tzinfo=datetime.timezone.utc))]))
CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='tell_a_joke', args={'previous_jokes': []}, tool_call_id='pyd_ai_79f54daf28b44a87b269e00fab60beaa')], usage=RequestUsage(input_tokens=228, output_tokens=253, details={'thoughts_tokens': 235, 'text_prompt_tokens': 228}), model_name='gemini-2.5-flash', timestamp=datetime.datetime(2025, 9, 3, 22, 19, 9, 372284, tzinfo=datetime.timezone.utc), provider_name='google-gla', provider_details={'finish_reason': 'STOP'}, provider_respon

In [9]:
print(nodes[-1].data.output.model_dump_json(indent=2))

{
  "jokes": [
    "Why don't scientists trust atoms? Because they make up everything!",
    "I told my wife she was drawing her eyebrows too high. She looked surprised.",
    "What do you call a fake noodle? An impasta!"
  ],
  "numbers": [
    64,
    41,
    15,
    45,
    41
  ],
  "poem": "Three jokes I've shared, with laughter in mind,\nOf atoms, high brows, and noodles you'll find.\nThen numbers emerged, from a mystical hand,\nSixty-four, forty-one, fifteen, across the land.\nWith forty-five and forty-one, a pair anew,\nA whimsical mix, for me and for you."
}


In [12]:
print(nodes[-1].data.output.poem)

Three jokes I've shared, with laughter in mind,
Of atoms, high brows, and noodles you'll find.
Then numbers emerged, from a mystical hand,
Sixty-four, forty-one, fifteen, across the land.
With forty-five and forty-one, a pair anew,
A whimsical mix, for me and for you.


# MCP Agents

## API MCPs: Context7

In [13]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStreamableHTTP

context7_server = MCPServerStreamableHTTP(
    url ='https://mcp.context7.com/mcp',
    headers={'CONTEXT7_API_KEY': settings.CONTEXT7_API_KEY}
)  

mcp_agent = Agent(
    settings.MODEL_NAME, toolsets=[context7_server], 
    instructions="You are a helpful assistant that can answer questions about code, you use Context7 to search first for the library and then ask some questions to context7 about that library, based on the results, you give an answer to the user"
)  
# Run the Agent
async with mcp_agent:  
    nodes = []
    async with mcp_agent.iter(
        "Give me the code for a simple agent that is connected to an MCP, using ",
    ) as agent_run:
        async for node in agent_run:
            # Each node represents a step in the agent's execution
            print(node)
            nodes.append(node)

UserPromptNode(user_prompt='Give me the code for a simple agent that is connected to an MCP, using ', instructions='You are a helpful assistant that can answer questions about code, you use Context7 to search first for the library and then ask some questions to context7 about that library, based on the results, you give an answer to the user', instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})
ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='Give me the code for a simple agent that is connected to an MCP, using ', timestamp=datetime.datetime(2025, 9, 3, 22, 20, 33, 862527, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that can answer questions about code, you use Context7 to search first for the library and then ask some questions to context7 about that library, based on the results, you give an answer to the user'))
CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(

In [17]:
print(nodes[-1].data.output)

Here is the code for a simple agent connected to an MCP, using the `mcp-use` library. This example demonstrates how to initialize an `MCPAgent` with a `MCPClient` and an OpenAI LLM, and then run a simple query. It also shows how to restrict certain tools for security or task-specific reasons.



```python
import asyncio
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from mcp_use import MCPAgent, MCPClient

async def main():
    # Load environment variables (e.g., OPENAI_API_KEY)
    load_dotenv()

    # Create a configuration dictionary for your MCP servers.
    # This example configures a 'playwright' server for web automation.
    config = {
      "mcpServers": {
        "playwright": {
          "command": "npx",
          "args": ["@playwright/mcp@latest"],
          "env": {
            "DISPLAY": ":1" # This might be needed for Playwright in some environments
          }
        }
      }
    }

    # Create an MCPClient from the configuration di

## Command MCPs: DuckDuckGo

In [18]:
from pydantic_ai.mcp import MCPServerStdio

ddg_server = MCPServerStdio(  
    'uvx',
    args=[
        'duckduckgo-mcp-server',
    ]
)
mcp_agent = Agent(settings.MODEL_NAME, toolsets=[ddg_server], 
                  instructions="You are a helpful assistant that can use the DuckDuckGo MCP server to search the web and obtain current information, you always search first and then fetch the content of the search results you think are relevant to the question, finally you answer the question based on the content of the search results")  
# Run the Agent
async with mcp_agent:  
    nodes = []
    async with mcp_agent.iter(
        "What is the weather in Tokyo?",
    ) as agent_run:
        async for node in agent_run:
            # Each node represents a step in the agent's execution
            print(node)
            nodes.append(node)

UserPromptNode(user_prompt='What is the weather in Tokyo?', instructions='You are a helpful assistant that can use the DuckDuckGo MCP server to search the web and obtain current information, you always search first and then fetch the content of the search results you think are relevant to the question, finally you answer the question based on the content of the search results', instructions_functions=[], system_prompts=(), system_prompt_functions=[], system_prompt_dynamic_functions={})
ModelRequestNode(request=ModelRequest(parts=[UserPromptPart(content='What is the weather in Tokyo?', timestamp=datetime.datetime(2025, 9, 3, 22, 21, 23, 80422, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that can use the DuckDuckGo MCP server to search the web and obtain current information, you always search first and then fetch the content of the search results you think are relevant to the question, finally you answer the question based on the content of the search resul

In [19]:
print(nodes[-1].data.output)

The weather in Tokyo, Japan is currently 80°F and mostly cloudy with a RealFeel® of 91°F. There is a heavy rain warning for inundation. The wind is from the NE at 7 mph with gusts up to 10 mph. The air quality is fair.

For today, Thursday, September 4th, it will be cloudy, very humid, and not as hot, with occasional rain in the afternoon. The high will be 83°F. Tonight, there will be a thunderstorm around in the evening, otherwise, it will be cloudy and very humid with a low of 77°F.

Looking ahead, there are heavy afternoon thunderstorms expected on Friday, September 5th, with a high of 81°F and a low of 75°F.


## Conclusion

In this notebook, we explored the fundamentals of creating AI agents using Pydantic AI:

### Key Takeaways

1. **Simple Agents**: Created basic agents that can generate text responses with customizable instructions
2. **Structured Output**: Used Pydantic models to ensure agents return well-formatted, typed data instead of plain text
3. **Dependencies**: Implemented stateful agents that can maintain context (like previous jokes) between interactions
4. **Tool Integration**: Built agents that can use external tools (random number generation, joke creation) to perform complex tasks
5. **MCP Integration**: Connected agents to external services through Model Context Protocol (MCP) servers:
   - **API MCPs**: Used Context7 for code documentation and library search
   - **Command MCPs**: Used DuckDuckGo for real-time web search and content fetching

### What We Learned

- Pydantic AI makes it easy to build reliable agents with structured outputs
- Dependencies allow agents to maintain state and context across interactions
- Tools enable agents to perform actions beyond text generation
- MCP servers provide a standardized way to connect agents to external services
- Agents can automatically chain tool calls to complete complex, multi-step tasks

This foundation prepares us for building more sophisticated agent architectures in the following notebooks.
